In [146]:
import wrf
import matplotlib
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import cartopy.crs as crs
import matplotlib.pyplot as plt
import matplotlib.image as image
import matplotlib.colors as colors
from metpy.plots import USCOUNTIES
from cartopy.feature import NaturalEarthFeature
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim, cartopy_ylim, latlon_coords)
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import rcParams
from pathlib import Path
rcParams['font.family']     = 'sans-serif'
rcParams['font.sans-serif'] = 'Helvetica'

In [147]:
pd.read_pickle('/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/data/wrf-extracted/single-pixel/wrf-sauc-2003-2010.pkl')

,DFGDP,T2,Q2,PSFC,U10,V10,LWUPB,LWDNB,SWUPB,SWDNB,RH
times,,,,,,,,,,,
2002-07-01 00:00:00,0.0,289.731750,0.010616,98279.164062,1.873486,2.345785,0.000000,0.000000,0.000000,0.000000,87.381705
2002-07-01 01:00:00,0.0,291.919373,0.010424,98232.500000,-0.117902,0.064035,459.276794,335.334930,55.103020,378.191711,74.644099
2002-07-01 02:00:00,0.0,291.787201,0.010821,98184.156250,1.832111,-1.582516,441.950989,323.332855,31.983067,212.462540,78.096859
2002-07-01 03:00:00,0.0,290.572174,0.009789,98161.484375,1.949642,-1.759287,418.402618,314.136108,4.140750,25.994493,76.279150
2002-07-01 04:00:00,0.0,288.817505,0.009621,98170.554688,1.335406,-1.838828,401.677368,308.238983,0.000000,0.000000,83.881068
...,...,...,...,...,...,...,...,...,...,...,...
2010-06-30 19:00:00,0.0,293.716522,0.010593,98123.101562,3.286204,-1.692011,507.239960,327.437012,162.789703,1005.829773,67.710092
2010-06-30 20:00:00,0.0,294.190002,0.010580,98102.406250,3.755641,-2.816803,514.156921,324.362610,169.201492,1047.243530,65.654197
2010-06-30 21:00:00,0.0,294.389008,0.010270,98059.062500,4.147295,-3.793150,511.102875,326.432312,165.968567,1026.467041,62.922114


In [9]:
dir = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/data/geospatial/wrf-vertical-profile/'
fname = 'wrfout-2008-08-vertprofile.nc'

ds = xr.open_dataset(dir + fname)
#ncfile = nc.Dataset(dir + fname)
#len(ncfile.variables['Times'])
ds

<xarray.Dataset> Size: 39GB
Dimensions:                (Time: 96, south_north: 234, west_east: 258,
                            bottom_top: 54, bottom_top_stag: 55,
                            soil_layers_stag: 4, west_east_stag: 259,
                            south_north_stag: 235, snow_layers_stag: 3,
                            snso_layers_stag: 7)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 23MB ...
    XLONG                  (Time, south_north, west_east) float32 23MB ...
    XTIME                  (Time) datetime64[ns] 768B ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 23MB ...
    XLONG_U                (Time, south_north, west_east_stag) float32 23MB ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 23MB ...
    XLONG_V                (Time, south_north_stag, west_east) float32 23MB ...
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag,
                                snow_layers_stag, snso_layers_stag
Data variables: (12/285)
    Times                  (Time) |S19 2kB ...
    LU_INDEX               (Time, south_north, west_east) float32 23MB ...
    ZNU                    (Time, bottom_top) float32 21kB ...
    ZNW                    (Time, bottom_top_stag) float32 21kB ...
    ZS                     (Time, soil_layers_stag) float32 2kB ...
    DZS                    (Time, soil_layers_stag) float32 2kB ...
    ...                     ...
    PCB                    (Time, south_north, west_east) float32 23MB ...
    PC                     (Time, south_north, west_east) float32 23MB ...
    LANDMASK               (Time, south_north, west_east) float32 23MB ...
    LAKEMASK               (Time, south_north, west_east) float32 23MB ...
    SST                    (Time, south_north, west_east) float32 23MB ...
    SST_INPUT              (Time, south_north, west_east) float32 23MB ...
Attributes: (12/146)
    TITLE:                            OUTPUT FROM WRF V4.0.1 MODEL
    START_DATE:                      2008-08-15_00:00:00
    SIMULATION_START_DATE:           2008-07-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        259
    SOUTH-NORTH_GRID_DIMENSION:      235
    BOTTOM-TOP_GRID_DIMENSION:       55
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      0
    ETAC:                            0.0

In [10]:
wrf.getvar(ds, 'height')

TypeError: iteration over a 0-d array

# Export WRF Terrain Heights as Georeferenced Netcdf 

In [78]:
import xarray as xr
import rioxarray as rio
import netCDF4 as nc
import wrf

# Define input/output paths
data_dir = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/data/geospatial/wrf-vertical-profile/'
fname = 'wrfout-2008-08-vertprofile.nc'

fpath = data_dir + fname

outdir = '/Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/data/geospatial/'
outfname = 'wrf-terrain.tif'

# Open dataset
ds = xr.open_dataset(fpath)
ds

<xarray.Dataset> Size: 39GB
Dimensions:                (Time: 96, south_north: 234, west_east: 258,
                            bottom_top: 54, bottom_top_stag: 55,
                            soil_layers_stag: 4, west_east_stag: 259,
                            south_north_stag: 235, snow_layers_stag: 3,
                            snso_layers_stag: 7)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 23MB ...
    XLONG                  (Time, south_north, west_east) float32 23MB ...
    XTIME                  (Time) datetime64[ns] 768B ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 23MB ...
    XLONG_U                (Time, south_north, west_east_stag) float32 23MB ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 23MB ...
    XLONG_V                (Time, south_north_stag, west_east) float32 23MB ...
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag,
                                snow_layers_stag, snso_layers_stag
Data variables: (12/285)
    Times                  (Time) |S19 2kB ...
    LU_INDEX               (Time, south_north, west_east) float32 23MB ...
    ZNU                    (Time, bottom_top) float32 21kB ...
    ZNW                    (Time, bottom_top_stag) float32 21kB ...
    ZS                     (Time, soil_layers_stag) float32 2kB ...
    DZS                    (Time, soil_layers_stag) float32 2kB ...
    ...                     ...
    PCB                    (Time, south_north, west_east) float32 23MB ...
    PC                     (Time, south_north, west_east) float32 23MB ...
    LANDMASK               (Time, south_north, west_east) float32 23MB ...
    LAKEMASK               (Time, south_north, west_east) float32 23MB ...
    SST                    (Time, south_north, west_east) float32 23MB ...
    SST_INPUT              (Time, south_north, west_east) float32 23MB ...
Attributes: (12/146)
    TITLE:                            OUTPUT FROM WRF V4.0.1 MODEL
    START_DATE:                      2008-08-15_00:00:00
    SIMULATION_START_DATE:           2008-07-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        259
    SOUTH-NORTH_GRID_DIMENSION:      235
    BOTTOM-TOP_GRID_DIMENSION:       55
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      0
    ETAC:                            0.0

In [144]:
import rasterio
from rasterio.transform import from_origin
from wrf import getvar, latlon_coords
import xarray as xr
import numpy as np

# Open the WRF file
wrf_file = fpath
ncfile = nc.Dataset(wrf_file)

# Extract terrain height
ter = getvar(ncfile, "ter", timeidx=0)

# Get latitude and longitude
lats, lons = latlon_coords(ter)

# Define bounding box for Santa Cruz Island
lonmin, lonmax = -119.9401, -119.5049
latmin, latmax = 33.9487, 34.1

# Create a mask for the desired bounds
mask = (lons >= lonmin) & (lons <= lonmax) & (lats >= latmin) & (lats <= latmax)

# Get the row/column indices of the bounding box
rows, cols = np.where(mask)

# Define bounding box indices
row_min, row_max = rows.min(), rows.max()
col_min, col_max = cols.min(), cols.max()

# Subset the data
ter_subset = ter[row_min:row_max+1, col_min:col_max+1]
lats_subset = lats[row_min:row_max+1, col_min:col_max+1]
lons_subset = lons[row_min:row_max+1, col_min:col_max+1]

# Compute new resolution
res_x = np.abs(lons_subset[0, 1] - lons_subset[0, 0])  # Longitude resolution
res_y = np.abs(lats_subset[1, 0] - lats_subset[0, 0])  # Latitude resolution

# Define new transform with upper-left corner of subset
transform = from_origin(lons_subset.min(), lats_subset.max(), res_x, res_y)

# Define output GeoTIFF file
output_tif = outdir + outfname

# Save as GeoTIFF
with rasterio.open(
    output_tif,
    "w",
    driver="GTiff",
    height=ter_subset.shape[0],
    width=ter_subset.shape[1],
    count=1,
    dtype=ter.dtype,
    crs="EPSG:4326",  # Assuming WGS84 (change if needed)
    transform=transform,
) as dst:
    dst.write(np.flipud(ter_subset.values), 1)  # Flip the array vertically
print(f"Saved clipped terrain height to {output_tif}")


Saved clipped terrain height to /Users/patmccornack/Documents/ucsb_fog_project/_repositories/sci-wrf-analysis/data/geospatial/wrf-terrain.tif


# Aggregate to hourly datasets

In [192]:
root_dir = Path().resolve().parent
data_dir = root_dir / 'data' / 'observational' / 'clean-datasets'
df_fnames = ['sci-sauc-clean-2003-2008.csv', 'sci-upem-clean-2005-2010.csv']

out_dir = root_dir / 'data' / 'observational' / 'aggregated-datasets'
out_fname = ['sauc-hourly.csv', 'upem-hourly.csv']

for i in range(len(df_fnames)):
    df = pd.read_csv(data_dir / df_fnames[i], index_col='time (PST)')
    df = df.drop(df.columns[0], axis=1)
    df.index = pd.to_datetime(df.index)


    df_avg_cols = df[['air temperature (C)', 'relative humidity (%)', 'wind speed (m/s)', 'wind gust (m/s)', 'wind direction (deg)']].resample('h').mean()
    df_sum_cols = df[['fog', 'fog tips', 'rain (mm)']].resample('h').sum()
    df_hourly = pd.concat([df_sum_cols, df_avg_cols], axis=1)

    df_hourly.to_csv(out_dir / out_fname[i])



